# Purpose:
- Split local z-stacks and register
- use lamf_analysis cortical_zstack_parallel branch, if not merged yet.

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
from lamf_analysis.ophys import zstack
import tifffile
import re
from ScanImageTiffReader import ScanImageTiffReader
from importlib import reload
from glob import glob

from dask.distributed import Client
from dask import delayed, compute

c:\Users\jinho.kim\Anaconda3\envs\allenvisb\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\jinho.kim\Anaconda3\envs\allenvisb\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\jinho.kim\Anaconda3\envs\allenvisb\lib\site-packages\numpy\.libs\libopenblas.gk7gx5keq4f6uyo3p26ulgbqyhgqo7j4.gfortran-win_amd64.dll
c:\Users\jinho.kim\Anaconda3\envs\allenvisb\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
zstack_paths = [Path(r"\\allen\programs\mindscope\workgroups\learning\pilots\online_motion_correction\mouse_746542\1406177928_local_z_stack0.tiff"),
                Path(r"\\allen\programs\mindscope\workgroups\learning\pilots\online_motion_correction\mouse_746542\1406177928_local_z_stack1.tiff"),
                Path(r"\\allen\programs\mindscope\workgroups\learning\pilots\online_motion_correction\mouse_746542\1406177928_local_z_stack2.tiff"),
                Path(r"\\allen\programs\mindscope\workgroups\learning\pilots\online_motion_correction\mouse_746542\1406177928_local_z_stack3.tiff")]
output_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\online_motion_correction\mouse_746542\1406177928')
output_dir.mkdir(exist_ok=True, parents=True)

In [4]:
# using dask

def run_one_stack(zstack_path, output_dir):
    zstack_reg, channels_saved = zstack.register_local_zstack_from_raw_tif(zstack_path)
    for ch_ind, channel in enumerate(channels_saved):
        tifffile.imsave(output_dir / f"{zstack_path.stem}_reg_ch_{channel}.tif", zstack_reg[ch_ind])

client = Client()
tasks = [delayed(run_one_stack)(zstack_path, output_dir) for zstack_path in zstack_paths]
results = compute(*tasks)
client.close()

In [5]:
# data_dir = Path(r'\\meso1acq\data\1401033078')
data_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\online_motion_correction\mouse_746542\111324_746542')
mouse_id = 746542

session_id = data_dir.stem
# zstack_paths = data_dir.glob(f'{session_id}_local_z_stack*.tiff')
zstack_paths = data_dir.glob('*_local_z_stack*.tiff')

output_base_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\online_motion_correction')
output_dir = output_base_dir / f'mouse_{mouse_id}/{session_id}'
output_dir.mkdir(exist_ok=True, parents=True)

In [4]:
zstack_paths.

<generator object Path.glob at 0x000001AFD967D270>

In [6]:
# each file takes about 5 minutes to process
for zstack_path in zstack_paths:
    zstack_reg, channels_saved = zstack.register_local_zstack_from_raw_tif(zstack_path)
    for ch_ind, channel in enumerate(channels_saved):
        tifffile.imsave(output_dir / f"{zstack_path.stem}_reg_ch_{channel}.tif", zstack_reg[ch_ind])

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_44868\4015102887.py:5: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(output_dir / f"{zstack_path.stem}_reg_ch_{channel}.tif", zstack_reg[ch_ind])


In [85]:
reload(zstack)
zstack_path = zstack_paths[0]
zstack_reg, channels_saved = zstack.register_local_zstack_from_raw_tif(zstack_path)

In [92]:
# save each channel as a separate tiff
for ch_ind, channel in enumerate(channels_saved):
    tifffile.imsave(output_dir / f"{zstack_path.stem}_reg_ch_{channel}.tif", zstack_reg[ch_ind])

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_64560\3486107564.py:3: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(output_dir / f"{zstack_path.stem}_reg_ch_{channel}.tif", zstack_reg[ch_ind])
